## Fine-tuning a pretrained transformer model from Hugging Face for text classification using PyTorch.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm


In [1]:
# from datasets import load_dataset

In [2]:
dataset = load_dataset("emotion")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
dataset["train"][:3]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong'],
 'label': [0, 0, 3]}

### Tokenize the data

In [7]:
# from transformers import AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/Users/sm/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

In [10]:
tokenized_data = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [12]:
# The tokenized dataset contains "text" column, which we need to remove because the transformer model does not accept raw text as input

tokenized_data = tokenized_data.remove_columns(["text"])

In [13]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [14]:
# Rename column "label" to "labels" because the model expects "labels" column

tokenized_data = tokenized_data.rename_column("label", "labels")

In [15]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [16]:
tokenized_data.set_format("torch")

In [17]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [18]:
# Select a small fraction of the data for training and evaluation

small_train_data = tokenized_data["train"].shuffle(seed=42).select(range(1000))
small_val_data = tokenized_data["test"].shuffle(seed=42).select(range(1000))

In [19]:
small_train_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

## DataLoader

In [20]:
# from torch.utils.data import DataLoader

train_loader = DataLoader(small_train_data, shuffle=True, batch_size=8)
val_loader = DataLoader(small_val_data, batch_size=8)

## Getting a pretrained transformer model from Hugging Face

In [21]:
# from transformers import AutoModelForSequenceClassification

# We will replace the output layer of the pretrained model with a new layer, which will output 6 values
# The weights in the new layer will be adjusted when we train the modified model on new data

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [23]:
# from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [24]:
# import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [25]:
# dataloader iterates over the dataset and returns dict_items object, which we need to 
# transform into a regular dictionary

for batch in train_loader:
    print(batch.items())
    break

dict_items([('labels', tensor([4, 1, 0, 0, 1, 1, 1, 0])), ('input_ids', tensor([[  101, 10047,  3110,  ...,     0,     0,     0],
        [  101,  1045,  2514,  ...,     0,     0,     0],
        [  101, 10047,  5305,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  2156,  ...,     0,     0,     0],
        [  101,  1045,  2001,  ...,     0,     0,     0],
        [  101,  8909,  2066,  ...,     0,     0,     0]])), ('attention_mask', tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]))])


In [26]:
for batch in train_loader:
    print({k: v.to(device) for k, v in batch.items()})
    break

{'labels': tensor([4, 0, 0, 0, 1, 1, 1, 0]), 'input_ids': tensor([[ 101, 1045, 2318,  ...,    0,    0,    0],
        [ 101, 1045, 2514,  ...,    0,    0,    0],
        [ 101, 1045, 2514,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2572,  ...,    0,    0,    0],
        [ 101, 1045, 5683,  ...,    0,    0,    0],
        [ 101, 1045, 2514,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [27]:
# from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        

  0%|          | 0/375 [00:00<?, ?it/s]

In [28]:
model.eval()
for batch in val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    print("outputs: ", outputs)
    logits = outputs.logits
    print("logits: ", logits)
    predictions = torch.argmax(logits, dim=-1)
    print("predictions: ", predictions)
    break
    

outputs:  SequenceClassifierOutput(loss=tensor(0.1218), logits=tensor([[ 3.9552, -1.4628, -1.3047, -0.1477,  0.5281, -1.7380],
        [-1.2675,  4.0091, -0.3530, -1.5876, -0.9311, -0.8032],
        [-1.1374,  3.5222,  1.2953, -1.8349, -1.8817, -1.4354],
        [-1.0024,  4.1088,  0.4523, -1.8550, -1.4940, -1.5696],
        [ 3.9637, -1.3681, -1.1258,  0.8676, -0.8791, -1.6526],
        [-0.5630,  1.2790,  2.4399, -1.4360, -1.6700, -1.4027],
        [ 4.3816, -1.2579, -1.1618, -0.4273, -0.1583, -1.7490],
        [-0.5774, -1.2477, -1.6858,  2.6211,  0.9773, -0.6734]]), hidden_states=None, attentions=None)
logits:  tensor([[ 3.9552, -1.4628, -1.3047, -0.1477,  0.5281, -1.7380],
        [-1.2675,  4.0091, -0.3530, -1.5876, -0.9311, -0.8032],
        [-1.1374,  3.5222,  1.2953, -1.8349, -1.8817, -1.4354],
        [-1.0024,  4.1088,  0.4523, -1.8550, -1.4940, -1.5696],
        [ 3.9637, -1.3681, -1.1258,  0.8676, -0.8791, -1.6526],
        [-0.5630,  1.2790,  2.4399, -1.4360, -1.6700, -1.

In [31]:
progress_bar = tqdm(range(len(val_loader)))

from sklearn.metrics import accuracy_score, f1_score

model.eval()

accuracy = 0
acc_score = 0
f1 = 0

for batch in val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    accuracy += torch.sum(predictions == batch['labels'])/len(batch['labels'])
    acc_score += accuracy_score(batch['labels'], predictions)
    f1 += f1_score(batch['labels'], predictions, average='weighted') 
    progress_bar.update(1)
    
   
accuracy = accuracy/len(val_loader)
acc_score = acc_score/len(val_loader)
f1 = f1/len(val_loader)

print("accuracy: ", accuracy)
print("accuracy score: ", acc_score)
print('f1 score: ', f1)

  0%|          | 0/125 [00:00<?, ?it/s]

accuracy:  tensor(0.8630)
accuracy score:  0.863
f1 score:  0.8624829670329671
